# Indeed Job Scraper
create a general purpose job scraper

In [27]:
#import csv
from datetime import datetime
import requests
from bs4 import BeautifulSoup
import csv

### Setup the query and url

In [2]:
def get_url(position, location):
    """Generate url from position and location"""
    template = 'https://www.indeed.fr/jobs?q={}&l={}'
    position = position.replace(' ', '+')
    location = location.replace(' ', '+')
    url = template.format(position, location)
    return url

In [3]:
url = get_url('developpeur', '')
print(url)

https://www.indeed.fr/jobs?q=developpeur&l=


### Extract the html data

In [4]:
response = requests.get(url)

In [5]:
soup = BeautifulSoup(response.text, 'html.parser')

In [6]:
cards = soup.find_all('div', 'job_seen_beacon')

### Prototype avec un seul enregistrement

In [7]:
card = cards[0]

#### Séparation en 3 groupes d'un enregistrement

In [8]:
topCard = card.find('td','resultContent')

In [9]:
botCard = card.find('table','jobCardShelfContainer')

#### Titre du poste

In [10]:
job_title = topCard.find('h2','jobTitle').text
job_title

'nouveauDEVELOPPEUR SKYWISE (H/F)'

#### Nom de l'entreprise

In [11]:
company = topCard.find('div', 'companyInfo').find('span','companyName').text
company

'Airbus'

#### Adresse de l'entreprise

In [12]:
job_location = topCard.find('div', 'companyLocation').text #.get('data-rc-loc')
job_location # A revoir

'31300 Toulouse'

#### Salaire proposé

In [13]:
salary = topCard.find('div','salary-snippet') # Dans le cas d'une offre avec le salaire precise

# Le salaire n'est pas toujours present sur les offres...
if salary:
    salary = salary.text.replace(u'\xa0', u' ')
else:
    salary = '' 
    
salary

''

#### Description du poste

In [14]:
summary = botCard.find('div','job-snippet').text.strip().replace('\n', ' ')
summary

'Vous souhaitez contribuer au développement de projets digitaux, de logiciels et de nouvelles applications ? Vous souhaitez contribuer à la croissance de notre…'

#### Date de publication

In [100]:
post_date = botCard.find('span', 'date').text
#today = datetime.today().strftime('%Y-%m-%d')

In [101]:
post_date

'Postedil y a 1 jour'

#### Final

In [110]:
record = (job_title, company, job_location,salary, summary,post_date)

In [111]:
record

('Developpeur Web H/F',
 'Avisto',
 'Télétravail in 06220 Vallauris',
 '42 000 € par an',
 "Acteur majeur et indépendant dans la profession et fort de 40 ans d'expérience, ACD propose des logiciels toujours plus performants et innovants afin d…",
 'Postedil y a 1 jour')

### Automatisation avec une classe

In [3]:
""" 
Author: Victorien
Date: 8/02  
Description:
    - Scrapping indeed en fonction d'un poste et/ou d'une adresse
    - creation d'une liste des jobs & infos
    - TODO : Export JSON ?
    - TODO : Export mongoDB ?
    
"""
class Scrapper():

    template = 'https://www.indeed.fr/jobs?q={}&l={}' # Constante de classe

    # constructeur
    def __init__(self,position='',location=''):
        
        self.position = position
        self.location = location
        self.records = [] # init de la liste des enregistrements
        
        url = self.getUrl(self.template)
        
        # Parcours de toutes les pages
        while True:
            response = requests.get(url)
            soup = BeautifulSoup(response.text, 'html.parser')
            self.cards = soup.find_all('div', 'job_seen_beacon') 

            # Construction du jeu d'enregistrement
            for card in cards:
                record = self.getRecord(card)
                self.records.append(record)
            try:
                url = 'https://www.indeed.fr' + soup.find('a', {'aria-label': 'Next'}).get('href')
            except AttributeError:
                break
        
    def getUrl(self,template):
        position = self.position.replace(' ', '+')
        location = self.location.replace(' ', '+')
        url = template.format(position, location)
        return url
    
    def getRecord(self,card):
        # Separation d'une carte indeed en 2 parties
        topCard = card.find('td','resultContent')
        botCard = card.find('table','jobCardShelfContainer')
        
        job_title = topCard.find('h2','jobTitle').text # Titre du poste
        company = topCard.find('div', 'companyInfo').find('span','companyName').text # Nom de l'entreprise
        job_location = topCard.find('div', 'companyLocation').text # localisation
        
        salary = topCard.find('div','salary-snippet') # Dans le cas d'une offre avec le salaire precise
        if salary: # Le salaire n'est pas toujours present sur les offres...
            salary = salary.text.replace(u'\xa0', u' ')
        else:
            salary = ''
            
        summary = botCard.find('div','job-snippet').text.strip().replace('\n', ' ') # Description du poste
        post_date = botCard.find('span', 'date').text # Date de publication
        
        record = (job_title, company, job_location,salary, summary,post_date)
        return record
    
    def exportCSV(self):
        # save the job data
        with open('results.csv', 'w', newline='', encoding='ISO-8859-1') as f:
            writer = csv.writer(f)
            # Titre
            writer.writerow(['Titre du poste', 'Entreprise', 'Adresse', 'Salaire','Decription','date'])
            # Ecriture
            writer.writerows(self.records)
            
    #def exportJSON():
        
    #def mongoDB():
        
            

In [35]:
scrap = Scrapper()

In [36]:
scrap.exportCSV()

In [27]:
# run the main program
main('senior accountant', 'charlotte nc')